# Statistics
## Run statistics

In [2]:
%autoreload 2 
import numpy as np 
from pianoq.misc.mplt import mplot, mimshow
from pianoq.simulations.abstract_quantum_scaling.slm3_simulation import QWFSSimulation
from pianoq.misc.mplc_writeup_imports import * 


In [ ]:
N_modes = 64
s = QWFSSimulation(N=N_modes)

# algos = ['slsqp', "L-BFGS-B", 'simulated_annealing', 'genetic_algorithm', 'PSO']
algos = ['slsqp', "L-BFGS-B", 'simulated_annealing']
configs = ['SLM1', 'SLM2', 'SLM3']
# configs = ['SLM1']
T_methods = ['unitary', 'gaus_iid']

N_tries = 100

results, Ts, best_phases = s.statistics(algos=algos, configs=configs, T_methods=T_methods, N_tries=N_tries)

# results = np.zeros((N_T_methods, N_configs, N_tries, N_algos))

np.save(rf'C:\temp\{tnow()}_qwfs_results.npy', results)

## Show results 

In [4]:
results = np.load(r'C:\temp\2024_11_12_20_23_30_qwfs_results.npy')
algos = ['slsqp', "L-BFGS-B", 'simulated_annealing']
configs = ['SLM1', 'SLM2', 'SLM3']
# configs = ['SLM1']
T_methods = ['unitary', 'gaus_iid']
N_tries = 100

In [5]:
results[0, 0]

array([[0.76482964, 0.76532619, 0.76531404],
       [0.84111476, 0.84122321, 0.84114461],
       [0.80832086, 0.80832859, 0.80835752],
       [0.7800289 , 0.78003977, 0.78003976],
       [0.81555082, 0.81556258, 0.81555312],
       [0.77489554, 0.77497648, 0.77497649],
       [0.82082817, 0.82087968, 0.82087968],
       [0.7645589 , 0.76458494, 0.76459514],
       [0.78350761, 0.7837889 , 0.78378888],
       [0.80733148, 0.80695805, 0.80737878],
       [0.80939207, 0.8094015 , 0.80940152],
       [0.84353077, 0.84343612, 0.84356345],
       [0.8073006 , 0.80782655, 0.80759662],
       [0.79174587, 0.79178644, 0.79178518],
       [0.81752503, 0.8175948 , 0.81728741],
       [0.79279608, 0.78689922, 0.79282352],
       [0.81145208, 0.81146632, 0.81131072],
       [0.77441259, 0.77442158, 0.77424421],
       [0.76778413, 0.76811699, 0.76805659],
       [0.78940954, 0.7898704 , 0.79006399],
       [0.78605055, 0.78605347, 0.78605347],
       [0.76273461, 0.76224944, 0.76249514],
       [0.

In [7]:
# all configurations 
fig, axes = plt.subplots(len(configs), len(T_methods))
for config_no, config in enumerate(configs):
    for T_method_no, T_method in enumerate(T_methods):
        # upper_lim = 1 if T_method == 'unitary' else 2 
        # imm = axes[config_no, T_method_no].imshow(results[T_method_no, config_no], clim=(0, upper_lim))
        if len(configs) > 1:
            ax = axes[config_no, T_method_no]
        else:
            ax = axes[T_method_no]
        imm = ax.imshow(results[T_method_no, config_no], aspect='auto')
        print(results[T_method_no, config_no].mean(axis=0))
        ax.set_title(rf'{config}, {T_method}')
        fig.colorbar(imm, ax=ax)
fig.show()

In [29]:
for config_no, config in enumerate(configs):
    print(f'---- {config} ----')
    for T_method_no, T_method in enumerate(T_methods):
        print(f'-- {T_method} --')
        for algo_no, algo in enumerate(algos):           
            avg = results[T_method_no, config_no].mean(axis=0)[algo_no]
            std = results[T_method_no, config_no].std(axis=0)[algo_no]
            
            print(f'{algo:<25} {avg:.2f}+-{std:.2f}')
    print()


---- SLM1 ----
-- unitary --
slsqp                     0.79+-0.03
L-BFGS-B                  0.79+-0.03
simulated_annealing       0.79+-0.03
-- gaus_iid --
slsqp                     0.81+-0.14
L-BFGS-B                  0.81+-0.14
simulated_annealing       0.81+-0.14

---- SLM2 ----
-- unitary --
slsqp                     0.61+-0.05
L-BFGS-B                  0.61+-0.05
simulated_annealing       0.61+-0.05
-- gaus_iid --
slsqp                     0.64+-0.12
L-BFGS-B                  0.64+-0.12
simulated_annealing       0.64+-0.12

---- SLM3 ----
-- unitary --
slsqp                     0.82+-0.03
L-BFGS-B                  0.83+-0.03
simulated_annealing       0.86+-0.02
-- gaus_iid --
slsqp                     1.58+-0.18
L-BFGS-B                  1.58+-0.17
simulated_annealing       1.73+-0.16



In [31]:
print(np.pi/4)
print((np.pi/4)**2)

0.7853981633974483
0.6168502750680849


In [26]:
# comparing whether they find the same phases 
mplot(best_phases[0, 2, 0, :3].T)

In [ ]:
for method, r in zip(algos, results.T):
    print(f'{method:<20} {r.mean():.3f} +- {r.std():.3f}')

In [153]:
s.propagate()
mplot(np.abs(v_out)**2)

(<Figure size 640x480 with 1 Axes>, <AxesSubplot: >)